<html>
<div style="background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: 600px; vertical-align: middle; height: 40px; margin: 10px;">
<h1 style="font-family: Georgia; color: black;">AI-Fall 01-CA2-Part2</h1>
</div>
<div style="background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: 500px; margin: 10px;">
  <img src="https://upload.wikimedia.org/wikipedia/en/thumb/f/fd/University_of_Tehran_logo.svg/225px-University_of_Tehran_logo.svg.png" width=60px width=auto style="padding:10px; vertical-align: middle;">
  <span style="font-family: Georgia; font-size:30px; color: black;">University of Tehran </span>
</div>
<div style=" background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: 400px; height: 30px; margin: 10px;">
  <span style="font-family: Georgia; font-size:15pt; color: black; vertical-align: middle;">Saman Eslami Nazari - std id: 810199375 </span>
</div>
</html>

First, I need a graph to represent the hexagon in the problem. Each edge must have a type to distinguish players.

In [280]:
class EdgeTypes:
    RED = "red"
    BLUE = "blue"
    IGNORE = "ignore"

In [281]:
from typing import Optional, Any


class Graph:
    class Edge:
        def __init__(self, u: int, v: int, edge_type: str):
            self.u = u
            self.v = v
            self.edge_type = edge_type

        def get_neighbor(self, u: int):
            if u == self.u:
                return self.v
            return self.u

        def __eq__(self, __o: object) -> bool:
            has_edge = (
                (self.u == __o.u and self.v == __o.v) or
                (self.u == __o.v and self.v == __o.u)
            )
            if __o.edge_type == EdgeTypes.IGNORE:
                return has_edge
            return has_edge and self.edge_type == __o.edge_type

    def __init__(self, nodes_cnt: Optional[int] = 6):
        self._nodes_cnt = nodes_cnt
        self._edges: list[Graph.Edge] = []

    def add_edge(self, u: int, v: int, edge_type: Any) -> None:
        self._edges.append(Graph.Edge(u, v, edge_type))

    def get_neighbors(self, u: int, edge_type: Any) -> list[int]:
        return [
            n.get_neighbor(u) for n in self._edges
            if (n.u == u or n.v == u) and n.edge_type == edge_type
        ]

    def has_cycle(self, cycle_len: int, edge_type: Any) -> bool:
        all_nodes = \
            [e.v for e in self._edges if e.edge_type == edge_type] + \
            [e.u for e in self._edges if e.edge_type == edge_type]

        for n in all_nodes:
            if self._has_cycle_recursive(
                n, edge_type, n, cycle_len, set()
            ):
                return True
        return False

    def has_edge(self, u: int, v: int, edge_type: Any) -> bool:
        return Graph.Edge(u, v, edge_type) in self._edges

    def get_empty_edges(self) -> list[tuple[int, int]]:
        result: list[tuple[int, int]] = []
        for i in range(self._nodes_cnt):
            for j in range(self._nodes_cnt):
                if i == j:
                    continue
                if self.has_edge(i, j, EdgeTypes.IGNORE):
                    continue
                result.append((i, j))
        return result

    def get_nodes(self) -> list[int]:
        return list(range(self._nodes_cnt))

    def get_node_degree(self, u: int, edge_type: Any) -> int:
        result: int = 0
        for e in self._edges:
            if e.u != u and e.v != u:
                continue
            if e.edge_type == edge_type:
                result += 1
        return result

    def get_edges(self, edge_type: Any) -> list[tuple[int, int]]:
        return [(e.u, e.v) for e in self._edges if e.edge_type == edge_type]

    def _has_cycle_recursive(
            self,
            current_node: int,
            edge_type: Any,
            init_node: int,
            cycle_len: int,
            explored_edges: set) -> bool:

        if cycle_len == 0:
            return False

        for next_node in self.get_neighbors(current_node, edge_type):
            if (
                (next_node, current_node) in explored_edges or
                (current_node, next_node) in explored_edges
            ):
                continue

            if next_node == init_node and cycle_len == 1:
                return True

            explored_edges.add((next_node, current_node))
            if self._has_cycle_recursive(
                    next_node, edge_type, init_node, cycle_len - 1, explored_edges):
                return True
            explored_edges.remove((next_node, current_node))

        return False


Now, lets create a `SimGame` class to simulate the game actions and states.

In [282]:
class SimGame:
    def __init__(self):
        self._graph = Graph()
    
    def put_line(self, u: int, v: int, color: str) -> None:
        self._graph.add_edge(u, v, color)

    def get_winner(self) -> str | None:
        if self._graph.has_cycle(3, EdgeTypes.RED):
            return EdgeTypes.BLUE
        if self._graph.has_cycle(3, EdgeTypes.BLUE):
            return EdgeTypes.RED
        return None
    
    def get_graph(self) -> Graph:
        return self._graph
    
    def get_available_lines(self) -> list[tuple[int, int]]:
        result = self._graph.get_empty_edges()
        return result

After this, I will make the `Agent` class that will decide its moves by creating a minimax tree.

In [283]:
from copy import deepcopy


class Agent:
    def __init__(self):
        self._game: SimGame = None

    def set_game(self, game: SimGame) -> None:
        self._game = game

    # returns the estimated final score of doing the move
    def decide_next_move(self, depth_limit: int, prune: Optional[bool] = False) -> None:
        _, best_move = self._max(self._game, depth_limit, prune)
        self._game.put_line(best_move[0], best_move[1], EdgeTypes.BLUE)

    def _min(self, game: SimGame, depth_limit: int, prune: Optional[bool] = False, prune_val: Optional[int] = -100000):
        if depth_limit == 0:
            return self._eval(game.get_graph()), None

        global_min = 10000
        best_line: tuple[int, int] = None
        for l in game.get_available_lines():
            game_cpy = deepcopy(game)
            game_cpy.put_line(l[0], l[1], EdgeTypes.RED)
            local_min = min(
                global_min,
                self._max(game_cpy, depth_limit - 1, prune, global_min)[0]
            )
            if local_min <= global_min:
                best_line = l
                global_min = local_min
            if prune and global_min <= prune_val:
                break
        return global_min, best_line

    def _max(self, game: SimGame, depth_limit: int, prune: Optional[bool] = False, prune_val: Optional[int] = 100000):
        if depth_limit == 0:
            return self._eval(game.get_graph()), None

        global_max = -10000
        best_line: tuple[int, int] = None
        for l in game.get_available_lines():
            game_cpy = deepcopy(game)
            game_cpy.put_line(l[0], l[1], EdgeTypes.BLUE)
            local_max = max(
                global_max,
                self._min(game_cpy, depth_limit - 1, prune, global_max)[0]
            )
            if local_max >= global_max:
                best_line = l
                global_max = local_max
            if prune and global_max >= prune_val:
                break
        return global_max, best_line

    def _eval(self, game_graph: Graph) -> int:
        result: int = 0
        for n in self._game.get_graph().get_nodes():
            result += self._game.get_graph().get_node_degree(n, EdgeTypes.RED) ** 2
            result -= self._game.get_graph().get_node_degree(n, EdgeTypes.BLUE) ** 2
        return result


I should also write a `RandomAgent` that will play against the `Agent`.

In [284]:
import random


class RandomAgent:
    def __init__(self):
        self._game: SimGame = None

    def set_game(self, game: SimGame) -> None:
        self._game = game

    def decide_next_move(self) -> None:
        choices = self._game.get_available_lines()
        chosen_line = random.choice(choices)
        self._game.put_line(chosen_line[0], chosen_line[1], EdgeTypes.RED)


Now lets create a simulation function that will put two agents against each other and return the winner. 

In [285]:
def simulate(depth_limit: int, blue_agent: Agent, red_agent: RandomAgent, game: SimGame, prune: bool) -> str:
    blue_agent.set_game(game)
    red_agent.set_game(game)
    while True:
        winner = None
        winner = game.get_winner()
        if winner != None:
            return winner
        blue_agent.decide_next_move(depth_limit, prune)
        winner = game.get_winner()
        if winner != None:
            return winner
        red_agent.decide_next_move()


To test the accuracy of our method, we need to run it about 100 to 200 times and see how many times blue succeeds.

In [288]:
def calc_win_chance(
    depth_limit: int,
    blue_agent: Agent,
    red_agent: RandomAgent,
    game: SimGame,
    run_cnt: int,
    prune: bool
) -> float:

    blue_wins = 0
    for _ in range(run_cnt):
        blue_agent_cpy = deepcopy(blue_agent)
        red_agent_cpy = deepcopy(red_agent)
        game_cpy = deepcopy(game)
        if simulate(depth_limit, blue_agent_cpy, red_agent_cpy, game_cpy, prune) == EdgeTypes.BLUE:
            blue_wins += 1
    return blue_wins / run_cnt

blue_agent = Agent()
red_agent = RandomAgent()
game = SimGame()
print(f"Win chance for level 1 without pruning: {calc_win_chance(1, blue_agent, red_agent, game, 100, False)}")
print(f"Win chance for level 1 with pruning: {calc_win_chance(1, blue_agent, red_agent, game, 100, True)}")
print(f"Win chance for level 1 without pruning: {calc_win_chance(3, blue_agent, red_agent, game, 100, False)}")
print(f"Win chance for level 1 with pruning: {calc_win_chance(3, blue_agent, red_agent, game, 100, True)}")

Win chance for level 1 without pruning: 0.28
Win chance for level 1 with pruning: 0.31
Win chance for level 1 without pruning: 0.38
Win chance for level 1 with pruning: 0.33


And in the final part, I shall write a function that will run and calculate te run time of our Agent.

In [287]:
from timeit import timeit
from textwrap import dedent


def test_agent():

    LEVELS_TO_TEST = [1, 3]
    SETUP = """
        from __main__ import simulate, Agent, RandomAgent, SimGame
        blue_agent = Agent()
        red_agent = RandomAgent()
        game = SimGame()
    """
    for l in LEVELS_TO_TEST:
        CODE = f"simulate({l}, blue_agent, red_agent, game, False)"
        print(
            f"Level {l} without pruning: {timeit(stmt=CODE, setup=dedent(SETUP), number=3)/3}"
        )
        PRUNE_CODE = f"simulate({l}, blue_agent, red_agent, game, True)"
        print(
            f"Level {l} with pruning: {timeit(stmt=PRUNE_CODE, setup=dedent(SETUP), number=3)/3}"
        )

test_agent()

Level 1 without pruning: 0.003989466664885792
Level 1 with pruning: 0.0037541000007574135
Level 3 without pruning: 1.4704068333327693
Level 3 with pruning: 0.09652526666711007
